# Regression Modelling

In [1]:
import numpy as np
import pandas as pd 
df = pd.read_csv("../data/COMP1801_Coursework_Dataset_Corrected.csv")
df.shape
df.head()

,Lifespan,partType,microstructure,coolingRate,quenchTime,forgeTime,HeatTreatTime,Nickel%,Iron%,Cobalt%,Chromium%,smallDefects,largeDefects,sliverDefects,seedLocation,castType
0,1107.81,Valve,singleGrain,27,2.64,8.08,23.35,60.65,19.76,15.18,4.41,7,0,0,Top,Investment
1,1226.13,Valve,colGrain,24,4.06,1.11,24.98,52.47,31.63,12.32,3.58,30,1,0,Top,Continuous
2,1914.12,Blade,singleGrain,20,4.36,8.54,34.71,60.98,26.56,11.60,0.86,22,0,0,Bottom,Continuous
3,1240.83,Valve,singleGrain,14,3.59,8.83,21.61,54.13,26.71,17.81,1.35,10,0,0,Bottom,Continuous
4,1599.40,Nozzle,singleGrain,22,3.06,3.04,29.65,62.88,24.06,12.08,0.98,21,0,0,Top,Investment


### Now we will split the dataset into train data and test data:

1. Train data contains **80%** of the dataset for training
2. Test data contains **20%** of the dataset purely for testing 

In [3]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Lifespan'])
y = df['Lifespan']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=17)
print(f"size of X_train is :{X_train.shape} and that of X_test is :{X_test.shape}")
print(f"size of y_train is :{y_train.shape} and that of y_test is :{y_test.shape}")

size of X_train is :(800, 15) and that of X_test is :(200, 15)
size of y_train is :(800,) and that of y_test is :(200,)


### Encoding features:

For encoding purpose I will choose OneHotEncoder to avoid imposing a false numerical order, which is a risk with the LabelEncoder.

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline

In [15]:
# grouping numerical and categorical feature columns

cat_col = X_train.select_dtypes(include=['object','category']).columns.tolist()
num_col = X_train.select_dtypes(include=['int64','float64']).columns.tolist()

print(cat_col)
print(num_col)

['partType', 'microstructure', 'seedLocation', 'castType']
['coolingRate', 'quenchTime', 'forgeTime', 'HeatTreatTime', 'Nickel%', 'Iron%', 'Cobalt%', 'Chromium%', 'smallDefects', 'largeDefects', 'sliverDefects']


In [16]:
# create preprocessing pipelines for each data type

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# Create preprocessing pipelines for each data type
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ],
    remainder='passthrough'
)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

num_feature_names = num_col
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(cat_col)

all_feature_names = list(num_feature_names) + list(cat_feature_names)

X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=all_feature_names, index=X_train.index)
X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=all_feature_names, index=X_test.index)

print(f"Original shape: {X_train.shape}")
print(f"Preprocessed shape: {X_train_preprocessed.shape}")
print(f"\nFeature names after preprocessing:\n{all_feature_names}")

Original shape: (800, 15)
Preprocessed shape: (800, 23)

Feature names after preprocessing:
['coolingRate', 'quenchTime', 'forgeTime', 'HeatTreatTime', 'Nickel%', 'Iron%', 'Cobalt%', 'Chromium%', 'smallDefects', 'largeDefects', 'sliverDefects', 'partType_Blade', 'partType_Block', 'partType_Nozzle', 'partType_Valve', 'microstructure_colGrain', 'microstructure_equiGrain', 'microstructure_singleGrain', 'seedLocation_Bottom', 'seedLocation_Top', 'castType_Continuous', 'castType_Die', 'castType_Investment']


---

### Base Modelling:

1. Now let us start with modelling our data with a baseline model that predicts mean of training data. 
2. This serves as a baseline for our final model which should outperform this dummy regressor.
3. This helps us identify if our model is learning anything at all.

In [18]:
from sklearn.dummy import DummyRegressor 
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVR 
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [19]:
# Baseline dummy regressor

baseline_model = DummyRegressor(strategy='mean')
baseline_model.fit(X_train_preprocessed, y_train)

y_train_baseline = baseline_model.predict(X_train_preprocessed)
y_test_baseline = baseline_model.predict(X_test_preprocessed)

baseline_train_r2 = r2_score(y_train, y_train_baseline)
baseline_test_r2 = r2_score(y_test, y_test_baseline)
baseline_test_mse = mean_squared_error(y_test, y_test_baseline)
baseline_test_mae = mean_absolute_error(y_test, y_test_baseline)

print(f"\nBaseline Model (Mean Strategy):")
print(f"  Train R²: {baseline_train_r2:.4f}")
print(f"  Test R²:  {baseline_test_r2:.4f}")
print(f"  Test MSE: {baseline_test_mse:.4f}")
print(f"  Test MAE: {baseline_test_mae:.4f}")


Baseline Model (Mean Strategy):
  Train R²: 0.0000
  Test R²:  -0.0000
  Test MSE: 113480.9922
  Test MAE: 279.3145


#### Key takeaways of baseline mode:

1. R2 score: model socres 0.0000 that means it cannot explain any variance at all since it uses 'mean' strategy
2. MSE and MAE tell us that the model is missing patterns by only predicting mean 